iMPORTS

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import os
from deep_translator import GoogleTranslator
import datetime

ModuleNotFoundError: No module named 'requests'

Utils

In [ ]:
def translation(source='pl', target='en'):
   return GoogleTranslator(source,target).translate(text)

In [ ]:
def extract(ancestor, selector=None, attribute = None, multiple = False):
    if selector:
        if multiple:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()

In [ ]:
selectors = {
    'opinion_id': (None, "data-entry-id"),
    'author': ("span.user-post__author-name",),
    'recommendation': ("span.user-post__author-recomendation > em",),
    'score': ("span.user-post__score-count",),
    'content_pl': ("div.user-post__text",),
    'pros_pl': ("div.review-feature__item--positive", None, True),
    'cons_pl': ("div.review-feature__item--negative", None, True),
    'thumbs_up': ("button.vote-yes", "data-total-vote",),
    'thumbs_down': ("button.vote-no", "data-total-vote",),
    'date_published': ("span.user-post__published > time:nth-child(1)", "datetime"),
    'date_purchased': ("span.user-post__published > time:nth-child(2)", "datetime")
}

In [ ]:
with open("./cookie.json", "r", encoding="UTF-8") as json_file:
    headers = json.load(json_file)
requests_url = "https://www.ceneo.pl/"
product_id = "157097123"
url = f"{requests_url}{product_id}#tab=reviews"
response = requests.get(url, headers = headers)

1. send a request for accessing first webpage with opinions about products


2. If responce is OK, parse HTML page content into DOM structure 

3. Extract from DOM structure  opinions and their components

4. Assign opinions with their cmponents to complex data structures


5. If there are more pages with opinions, repeat all steps 1-5

In [ ]:
all_opinions = []
while url is not None:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        page_doc = BeautifulSoup(response.text, 'html.parser')
        opinions = page_doc.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *values) 
                for key, values in selectors.items()
            }

            
            try:
                single_opinion['content_en'] = translate(single_opinion['content_pl'])
                single_opinion['pros_en'] = translate(single_opinion['pros_pl'])
                single_opinion['cons_en'] = translate(single_opinion['cons_pl'])
            except Exception:
                single_opinion['content_en'] = None
                single_opinion['pros_en'] = None
                single_opinion['cons_en'] = None
            
            single_opinion['recommendation'] = True if single_opinion['recommendation'] == "polecam" else False if single_opinion['recommendation']  == "Nie polecam" else None
            single_opinion['score'] = float(single_opinion['score'].split('/')[0].replace(",","."))
            
            single_opinion['thumbs_up'] = int(single_opinion['thumbs_up'])
            single_opinion['thumbs__down'] = int(single_opinion['thumbs_down'])

            all_opinions.append(single_opinion)
        try:
            url = "https://www.ceneo.pl"+page_doc.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None

In [ ]:
for single_opinion in all_opinions:
    try:
        single_opinion['date_published'] = single_opinion['date_published'].strftime("%Y-%m-%d, %H:%M:%S") if single_opinion['date_published'] else None
    except AttributeError:
        pass
    try:
        single_opinion['date_purchased'] = single_opinion['date_purchased'].strftime("%Y-%m-%d, %H:%M:%S") if single_opinion['date_purchased'] else None
    except AttributeError:
        pass

In [ ]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as json_file:
    json.dump(all_opinions, json_file, indent=4, ensure_ascii=False)